In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import json
load_dotenv()

True

In [2]:
client = OpenAI()

In [18]:
from open_meteo_api import get_weather_data, get_air_data

In [5]:
# These are the functions to be called by the Assistant. These will be API calls to a weather app

# from open_meteo_apis import get_weather_data
# from open_meteo_apis import get_air_data

def get_current_temperature(latitude, longitude, unit):
    temp = get_weather_data({
        "latitude": latitude, 
        "longitude": longitude,
        "current": "temperature_2m"
    })
    return temp

def get_air_quality(latitude, longitude):
    air = get_air_data({
        "latitude": latitude, 
        "longitude": longitude
    })
    if air <= 20: return 'good'
    if air <= 40: return 'fair'
    if air <= 60: return 'moderate'
    if air <= 80: return 'poor'
    if air <= 100: return 'very poor'
    else: return 'extremely poor'
    

In [6]:
assistant = client.beta.assistants.create(
  name="Current Weather and Rain",
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4o",
  tools=[
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "latitude": {
              "type": "string",
              "description": "The latitute of the location."
            },
              "longitude": {
              "type": "string",
              "description": "The longitude of the location."
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["latitude", "longitude", "unit"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_air_quality",
        "description": "Get the air quality for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "latitude": {
              "type": "string",
              "description": "The latitute of the location."
            },
              "longitude": {
              "type": "string",
              "description": "The longitude of the location."
            },
          },
          "required": ["latitude", "longitude"]
        }
      }
    }
  ]
)

In [4]:
# If we want to discard the previously used thread:
client.beta.threads.delete(thread.id)

NameError: name 'thread' is not defined

In [7]:
# Create a thread
thread = client.beta.threads.create()

In [8]:
thread.id

'thread_Gt9Lnanc5Egshe4xMPnfXARz'

In [50]:
# Create our first message. Note that we ask for 2 things, hence 2 function calls are needed.
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What's the weather and the air quality in Beijing, China today?",
)

In [51]:
# Create a run without streaming
# Note that runs expire after 10 minutes.

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [52]:
run.status

'requires_action'

In [53]:
tool_call_1 = run.required_action.submit_tool_outputs.tool_calls[0]
arguments_1 = json.loads(tool_call_1.function.arguments)
function_name_1 = tool_call_1.function.name
print('Function 1: ', function_name_1)
print('Arguments 1: ', arguments_1, '\n')

tool_call_2 = run.required_action.submit_tool_outputs.tool_calls[1]
arguments_2 = json.loads(tool_call_2.function.arguments)
function_name_2 = tool_call_2.function.name
print('Function 2: ', function_name_2)
print('Arguments 2: ', arguments_2)

Function 1:  get_current_temperature
Arguments 1:  {'latitude': '39.9042', 'longitude': '116.4074', 'unit': 'Celsius'} 

Function 2:  get_air_quality
Arguments 2:  {'latitude': '39.9042', 'longitude': '116.4074'}


In [54]:
# Run the funcitons locally:
temp_result = get_current_temperature(arguments_1['latitude'],arguments_1['longitude'], arguments_1['unit'])
print('Temperature: ', temp_result)

air_quality_result = get_air_quality(arguments_2['latitude'], arguments_2['longitude'])
print('Air quality: ', air_quality_result)

Coordinates 39.875°N 116.375°E
Elevation 47.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Current time 1718568000
Current temperature_2m 23.600000381469727
Temperature:  23.600000381469727
Coordinates 40.0°N 116.39999389648438°E
Elevation 47.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Current: <openmeteo_sdk.VariablesWithTime.VariablesWithTime object at 0x127159a20>
Current time 1718568000
Air quality:  poor


In [55]:
# Define the list to store tool outputs
tool_outputs = []
 
# Loop through each tool in the required action section and append them to tool_outputs.
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_current_temperature":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": str(temp_result)
    })
  elif tool.function.name == "get_air_quality":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": air_quality_result
    })

In [56]:
tool_outputs

[{'tool_call_id': 'call_nLFEw8Mp0bjNIctWTJPZVna8',
  'output': '23.600000381469727'},
 {'tool_call_id': 'call_RfjRVpsQVodm7bDiZO0s8JtM', 'output': 'poor'}]

In [57]:
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")
 
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

Tool outputs submitted successfully.
SyncCursorPage[Message](data=[Message(id='msg_KOnktxheQEUZj2ZtPVemGzct', assistant_id='asst_7eepyoqadCncHO7pjJBInRB0', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Today's weather in Beijing, China is 23.6°C with poor air quality."), type='text')], created_at=1718568760, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_h7hFRL0uagVY1gvqSxqmZr5V', status=None, thread_id='thread_Gt9Lnanc5Egshe4xMPnfXARz'), Message(id='msg_AVCkMyIk5BG14Q3aTcoyQgYi', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="What's the weather and the air quality in Beijing, China today?"), type='text')], created_at=1718568740, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Gt9Lnanc5Egshe4xMPnfXARz'), Message(id='ms

In [58]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

In [59]:
for message in messages:
    print(message.content[0].text.value)
    print('\n')

What's the weather and the air quality in Ascot, UK today?


Today's weather in Ascot, UK is 14.8°C with fair air quality.


What's the weather and the air quality in Budapest today?


Today's weather in Budapest, Hungary is 20.6°C with fair air quality.


What's the weather and the air quality in Sunninghill, UK today?


Today's weather in Sunninghill, UK is 15.4°C with fair air quality.


What's the weather and the air quality in Beijing, China today?


Today's weather in Beijing, China is 23.6°C with poor air quality.




In [36]:
# Delete the assistant:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_hSM5zfbTEpoLYEwEjjLYwZrg', deleted=True, object='assistant.deleted')